## This notebook is used to analyse the intensity evaluation of lane-changing interactions in highD

In [ ]:
import sys
import numpy as np
from matplotlib import pyplot as plt
font = {'family' : 'Arial',
        'size'   : 9}
plt.rc('font', **font)
def to_grayscale(fig):
    fig.canvas.draw()
    img = np.array(fig.canvas.renderer.buffer_rgba())
    grayscale_img = np.dot(img[..., :3], [0.2989, 0.5870, 0.1140])
    fig_gray, ax_gray = plt.subplots(figsize=(fig.get_size_inches()), dpi=fig.dpi)
    ax_gray.imshow(grayscale_img, cmap=plt.get_cmap('gray'), vmin=0, vmax=255)
    ax_gray.axis('off')  # Turn off the axis
    ax_gray.set_title('Grayscale plot')
sys.path.append('../../')
from DataProcessing.utils.coortrans import coortrans
coortrans = coortrans()
from Demonstration.demonstration_utils import *


path_raw = '../../Data/RawData/'
path_processed = '../../Data/ProcessedData/'
path_input = '../../Data/InputData/'
path_output = '../../Data/OutputData/'
fig_path = r'C:/SURFdrive/PhD progress/PhDResearch/4_Conflict/AMAR/Figures/'

### Read data

In [ ]:
interaction_cases = pd.read_csv(path_output + 'intensity_evaluation/highD_interactive_LC.csv')
interaction_cases = interaction_cases.drop_duplicates(subset=['veh_id_i','frame_start','frame_end'])
interaction_cases = interaction_cases.set_index('lc_id')

conflict_cases = pd.read_csv(path_output + 'intensity_evaluation/highD_conflict_LC.csv')
conflict_cases = conflict_cases.drop_duplicates(subset=['lc_id','veh_id_i','veh_id_j'])
conflict_cases[['frame_start','frame_end']] = interaction_cases.loc[conflict_cases['lc_id']][['frame_start','frame_end']].values.astype(int)
conflict_cases['duration'] = conflict_cases['frame_end'] - conflict_cases['frame_start']
conflict_cases.loc[conflict_cases['avg_intensity'].isna(), 'avg_intensity'] = 1.
conflict_cases.loc[conflict_cases['avg_TTC'].isna(), 'avg_TTC'] = np.inf

conflicts = pd.read_hdf(path_output + 'intensity_evaluation/highD_conflict_LC_data.h5', key='data')
conflicts = conflicts.drop_duplicates(subset=['lc_id','veh_id_i','veh_id_j','frame_id'])
lane_changes = conflicts.groupby('lc_id')['frame_id'].apply(lambda x: x.between(interaction_cases.loc[x.name]['lc_start'], interaction_cases.loc[x.name]['lc_end'])).reset_index()
lane_changes = lane_changes.rename(columns={'level_1':'idx','frame_id':'lane_change'}).set_index('idx')

### Spatial distribution of target vehicles

In [ ]:
conflicts_view_i = coortrans.transform_coor(conflicts[lane_changes['lane_change']], view='i')
metadatafiles =  sorted(glob.glob(path_raw + 'highD/RecordingMetadata/*.csv'))
metadata = []
for metadatafile in metadatafiles:
    df = pd.read_csv(metadatafile)
    metadata.append(df)
metadata = pd.concat(metadata).set_index('id')

lane_width_list = []
for idx in metadata.index:
    lane_markings = [float(y) for lane in ['lowerLaneMarkings','upperLaneMarkings'] for y in metadata.loc[idx][lane].split(';')]
    lane_markings = np.sort(lane_markings)
    lane_widths = np.diff(lane_markings)
    lane_width_list.append(np.sort(lane_widths)[:-1])
lane_width_list = np.concatenate(lane_width_list)
avg_lane_width = np.mean(lane_width_list)


fig, axes = plt.subplots(1, 2, figsize=(7.5,1.8), sharex=True, sharey=True, constrained_layout=True)
for ax in axes:
    ax.plot([-70, 70], [avg_lane_width*0.5, avg_lane_width*0.5], 'k--', lw=0.5, alpha=0.2)
    ax.plot([-70, 70], [-avg_lane_width*0.5, -avg_lane_width*0.5], 'k--', lw=0.5, alpha=0.2)
    ax.plot([-70, 70], [avg_lane_width*1.5, avg_lane_width*1.5], 'k--', lw=0.5, alpha=0.2)
    ax.plot([-70, 70], [-avg_lane_width*1.5, -avg_lane_width*1.5], 'k--', lw=0.5, alpha=0.2)

test = conflicts_view_i.loc[conflicts_view_i.groupby('lc_id')['TTC'].idxmin()]
test = test[test['TTC']<4.2].sort_values('TTC')
axes[0].scatter(test['y_j'], test['x_j'], marker='s', s=15, lw=0.6,
                edgecolors=mpl.cm.YlOrRd_r(test['TTC']/4.2), facecolors='none')
axes[0].arrow(-50, -4.5, 7, 0, head_width=1.5, head_length=4, fc='r', ec='r')
axes[0].text(-45, -avg_lane_width*3, 'Heading direction\nof the ego vehicle', fontsize=8, ha='center', va='bottom')
fig.colorbar(plt.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin=-0.2, vmax=4.2), cmap='YlOrRd_r'), ax=axes[0], pad=0.02, shrink=0.5, aspect=8, ticks=[0, 1, 2, 3, 4], label='TTC (s)')
axes[0].set_xlim(-70, 70)
axes[0].set_ylim(-avg_lane_width*3, avg_lane_width*3)
axes[0].set_xlabel('Longitudinal position (m)')
axes[0].set_ylabel('Lateral position (m)')
axes[0].set_aspect(1.8)
axes[0].set_title('Spatial distribution of min. TTC in every lane-change', fontsize=9)

test = conflicts_view_i.loc[conflicts_view_i.groupby('lc_id')['n_hat'].idxmax()]
test = test[test['n_hat']>17].sort_values('n_hat')
axes[1].scatter(test['y_j'], test['x_j'], marker='s', s=15, lw=0.6,
                edgecolors=mpl.cm.YlOrRd((np.log10(test['n_hat'])-1.23)/(4.77-1.23)), facecolors='none')
cbr = fig.colorbar(plt.cm.ScalarMappable(norm=mpl.colors.LogNorm(vmin=17, vmax=10**4.2), cmap='YlOrRd'), ax=axes[1], pad=0.02, shrink=0.5, aspect=8, label='Unified intensity')
cbr.formatter = mpl.ticker.LogFormatterExponent(base=10)
cbr.set_ticks([10**i for i in range(2,5)])
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Arial'
plt.rcParams['mathtext.it'] = 'Arial:italic'
cbr.ax.set_yticklabels([r'$10^{%d}$' % i for i in range(2,5)], fontsize=8, fontname='Arial')
axes[1].set_xlabel('Longitudinal position (m)')
axes[1].set_aspect(1.8)
axes[1].set_title('Spatial distribution of max. intensity in every lane-change', fontsize=9)

to_grayscale(fig)

In [ ]:
fig.savefig(fig_path + 'highD_conflict_distribution.pdf', dpi=600, bbox_inches='tight')

### Summary of intensity evaluation

In [ ]:
num_interactions = interaction_cases.index.nunique()
print(f'There are {num_interactions} lane-changes in the highD dataset.')
counts = conflict_cases.groupby('lc_id')['conflict_id'].agg(['size']).groupby('size').size()
print(f'{counts[1]} lane-changes conflict with 1 vehicle, {counts[2]} lane-changes conflict with 2 vehicles, and {counts[3]} lane-changes conflict with 3 vehicles.')
print(f'In total {conflict_cases['lc_id'].nunique()}, {conflict_cases['lc_id'].nunique()/num_interactions*100:.2f}% of lane-changes conflict with other vehicles.')

In [ ]:
num_conflicts = conflict_cases.shape[0]
print(f'In these {num_conflicts} conflicts, where at least 1 second is continuously identified,')

num_conflicts = conflict_cases[(conflict_cases['TTC'])&(conflict_cases['Unified'])].shape[0]
print(f'{num_conflicts} are identified by both TTC and Unified')

num_conflicts = conflict_cases[(conflict_cases['TTC'])&(~conflict_cases['Unified'])].shape[0]
print(f'{num_conflicts} are identified by TTC only')

num_conflicts = conflict_cases[(~conflict_cases['TTC'])&(conflict_cases['Unified'])].shape[0]
print(f'{num_conflicts} are identified by Unified only')

In [ ]:
intensity_lower = np.arange(0, conflict_cases['avg_intensity'].max(), 0.5)
intensity_upper = intensity_lower+0.5
folder_names = [f'intensity_{l}-{u}' for l,u in zip(intensity_lower, intensity_upper)]
features2save = ['location','veh_id_i','veh_id_j','frame_start','frame_end','avg_intensity','avg_TTC']

up_down = conflicts[lane_changes['lane_change']].groupby('conflict_id')['n_hat'].agg(['first','last','mean','median'])
up_down = up_down[(up_down['first']<up_down['mean'])&(up_down['last']<up_down['median'])]

summary_table = []
for folder_name, intensity_l, intensity_u in zip(folder_names, intensity_lower, intensity_upper):
    summary_df = pd.DataFrame([[0.,0.,'', 0, 0, 0, 0, np.nan, np.nan]]*3, 
                              columns=['intensity_lower','intensity_upper','specification','num_conflicts','location','veh_id_i','veh_id_j','avg_intensity','avg_TTC'])
    summary_df['intensity_lower'] = intensity_l
    summary_df['intensity_upper'] = intensity_u
    use_cases = conflict_cases[(conflict_cases['avg_intensity']>=intensity_l)&(conflict_cases['avg_intensity']<intensity_u)].copy()

    summary_df.loc[0,'specification'] = 'Both TTC and Unified'
    use_case = use_cases[(use_cases['TTC'])&(use_cases['Unified'])]
    summary_df.loc[0,'num_conflicts'] = len(use_case)
    if summary_df.loc[0,'num_conflicts'] > 0:
        use_case = use_case[use_case['conflict_id'].isin(up_down.index)]
        if len(use_case) == 0:
            use_case = use_cases[(use_cases['TTC'])&(use_cases['Unified'])]
        dis2median = np.sum((use_case[['avg_intensity','avg_TTC']] - use_case[['avg_intensity','avg_TTC']].median())**2, axis=1)
        summary_df.loc[0, features2save] = use_case.loc[dis2median.idxmin()][features2save].values

    summary_df.loc[1,'specification'] = 'TTC only'
    use_case = use_cases[(use_cases['TTC'])&(~use_cases['Unified'])]
    summary_df.loc[1,'num_conflicts'] = len(use_case)
    if summary_df.loc[1,'num_conflicts'] > 0:
        use_case = use_case[use_case['conflict_id'].isin(up_down.index)]
        if len(use_case) == 0:
            use_case = use_cases[(use_cases['TTC'])&(~use_cases['Unified'])]
        dist2median = np.sum((use_case[['avg_intensity','avg_TTC']] - use_case[['avg_intensity','avg_TTC']].median())**2, axis=1)
        summary_df.loc[1, features2save] = use_case.loc[dist2median.idxmin()][features2save].values

    summary_df.loc[2,'specification'] = 'Unified only'
    use_case = use_cases[(~use_cases['TTC'])&(use_cases['Unified'])]
    summary_df.loc[2,'num_conflicts'] = len(use_case)
    if summary_df.loc[2,'num_conflicts'] > 0:
        use_case = use_case[use_case['conflict_id'].isin(up_down.index)]
        if len(use_case) == 0:
            use_case = use_cases[(~use_cases['TTC'])&(use_cases['Unified'])]
        dist2median = np.sum((use_case[['avg_intensity','avg_TTC']] - use_case[['avg_intensity','avg_TTC']].median())**2, axis=1)
        summary_df.loc[2, features2save] = use_case.loc[dist2median.idxmin()][features2save].values

    summary_table.append(summary_df)

summary_table = pd.concat(summary_table).reset_index(drop=True)
summary_table.to_csv(path_output + 'intensity_evaluation/highD_conflict_LC_to_visualse.csv', index=False)

In [ ]:
summary_table.sort_values(by=['specification','intensity_lower'])

In [ ]:
summary_table.groupby('specification')['num_conflicts'].sum(), summary_table.groupby('specification')['num_conflicts'].sum().sum()

In [ ]:
summary_table.groupby(['intensity_lower','intensity_upper'])['num_conflicts'].sum(), summary_table.groupby(['intensity_lower','intensity_upper'])['num_conflicts'].sum().sum()

### Power-law distribution of intensity

In [ ]:
def rgb_color(color, alpha=1.):
    return (color[0], color[1], color[2], alpha)

def trendline(x, y):
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    return p
    
plt.rcParams['mathtext.fontset'] = 'stix'
fig, axes = plt.subplots(1,5,figsize=(7.5,1.45), constrained_layout=True, gridspec_kw={'width_ratios': [1,1,0.05,1,1]})
axes[2].axis('off')
cmap = mpl.cm.plasma

conflict_cases['avg_TTC'].hist(bins=np.linspace(0.2, 4.2, 21), 
                               fc=rgb_color(cmap(0.), 0.3), ec=rgb_color(cmap(0.), 0.9), 
                               lw=0.5, label='Averaged during lane-change conflicts', grid=False, ax=axes[0])
ax0 = axes[0].twinx()
conflicts['TTC'].hist(bins=np.linspace(0.2, 4.2, 21), 
                      fc=rgb_color(cmap(0.6), 0.3), ec=rgb_color(cmap(0.6), 0.9), 
                      lw=0.5, label='Individual moments', grid=False, ax=ax0)
axes[0].set_xlabel('TTC (s)', labelpad=5)
axes[0].set_ylabel('Case frequency', labelpad=1)
ax0.set_ylabel('Moment frequency', labelpad=1)
axes[0].yaxis.label.set_color(cmap(0.))
ax0.yaxis.label.set_color(cmap(0.6))
axes[0].tick_params(axis='y', colors=cmap(0.), labelsize=8, pad=1)
ax0.tick_params(axis='y', colors=cmap(0.6), labelsize=8, pad=1)
axes[0].set_xlim(-0.1, 4.1)
axes[0].set_xticks([0,1,2,3,4])

axes[1].set_xscale('log')
(10**conflict_cases['avg_intensity']).hist(bins=[10**i for i in np.linspace(np.log10(17),3.5,21)],
                                     fc=rgb_color(cmap(0.), 0.3), ec=rgb_color(cmap(0.), 0.9),
                                     lw=0.5, label='Averaged during lane-change conflicts', grid=False, ax=axes[1])
ax1 = axes[1].twinx()
conflicts['n_hat'].hist(bins=[10**i for i in np.linspace(np.log10(17),3.5,21)],
                        fc=rgb_color(cmap(0.6), 0.3), ec=rgb_color(cmap(0.6), 0.9),
                        lw=0.5, label='Individual moments', grid=False, ax=ax1)
axes[1].set_xlabel('Unified intensity', labelpad=0)
axes[1].set_ylabel('Case frequency', labelpad=1)
ax1.set_ylabel('Moment frequency', labelpad=1)
axes[1].yaxis.label.set_color(cmap(0.))
ax1.yaxis.label.set_color(cmap(0.6))
axes[1].tick_params(axis='y', colors=cmap(0.), labelsize=8, pad=1)
ax1.tick_params(axis='y', colors=cmap(0.6), labelsize=8, pad=1)

handle1, label1 = axes[0].get_legend_handles_labels()
handle2, label2 = ax0.get_legend_handles_labels()
ax_legend = fig.add_axes([0., 0., 0.5, 1])
ax_legend.axis('off')
ax_legend.legend(handle1+handle2, label1+label2, ncol=2, bbox_to_anchor=(0.5, -0.1), loc='lower center',
                 fontsize=8, frameon=False, handlelength=1., handletextpad=0.3, borderaxespad=0.3, borderpad=0.3)
ax_legend.set_title('Histograms of TTC and Unified intensity', fontsize=9)


bins = [10**i for i in np.linspace(np.log10(0.15),np.log10(10),41)]
ax3 = axes[3].twinx()
hist, bins = np.histogram(conflicts['TTC'], bins=bins)
bins = (bins[:-1]+bins[1:])/2
ax3.scatter(bins, hist, s=10, color=cmap(0.6), marker='x', lw=0.8, alpha=0.6, label='Individual moments')
bins = [10**i for i in np.linspace(np.log10(0.15),np.log10(10),41)]
hist, bins = np.histogram(conflict_cases['avg_TTC'], bins=bins)
bins = (bins[:-1]+bins[1:])/2
axes[3].scatter(bins, hist, s=15, color=cmap(0.), marker='+', lw=0.8, alpha=0.6, label='Averaged during lane-change conflicts')
axes[3].loglog()
ax3.loglog()
ylim = ax3.get_ylim()
ax3.set_ylim(ylim[0], ylim[1])
ylim = axes[3].get_ylim()
axes[3].set_ylim(ylim[0], ylim[1])
axes[3].set_xlabel('TTC (s)', labelpad=0)
axes[3].set_ylabel('Case frequency', labelpad=1)
ax3.set_ylabel('Moment frequency', labelpad=1)
axes[3].yaxis.label.set_color(cmap(0.))
ax3.yaxis.label.set_color(cmap(0.6))
axes[3].tick_params(axis='y', which='both', colors=cmap(0.), labelsize=8, pad=1)
ax3.tick_params(axis='y', which='both', colors=cmap(0.6), labelsize=8, pad=1)
hist_conflict_cases = hist.copy()


bins = [10**i for i in np.linspace(np.log10(0.15),np.log10(10),41)]
hist, bins = np.histogram(conflicts['TTC'], bins=bins)
bins = (bins[:-1]+bins[1:])/2
bins, hist = bins[hist_conflict_cases>0], hist[hist_conflict_cases>0]
tl = trendline(np.log10(bins), np.log10(hist))
bins = [10**i for i in np.linspace(np.log10(0.15),np.log10(10),41)]
ax3.plot(bins, 10**tl(np.log10(bins)), ls='--', color=cmap(0.6), lw=0.8)
ax3.text(0.02, 0.98, "$y=10^{%.1f}x^{%.1f}$"%(tl[0],tl[1]), fontsize=8, ha='left', va='top', transform=ax3.transAxes, color=cmap(0.6))
bins = [10**i for i in np.linspace(np.log10(0.15),np.log10(10),41)]
hist, bins = np.histogram(conflict_cases['avg_TTC'], bins=bins)
bins = (bins[:-1]+bins[1:])/2
bins, hist = bins[hist_conflict_cases>0], hist_conflict_cases[hist_conflict_cases>0]
tl = trendline(np.log10(bins), np.log10(hist))
bins = [10**i for i in np.linspace(np.log10(0.15),np.log10(10),41)]
axes[3].plot(bins, 10**tl(np.log10(bins)), ls='--', color=cmap(0.), lw=0.8)
axes[3].text(0.02, 0.85, "$y=10^{%.1f}x^{%.1f}$"%(tl[0],tl[1]), fontsize=8, ha='left', va='top', transform=axes[3].transAxes, color=cmap(0.))


bins = [10**i for i in np.linspace(np.log10(1.1), 3.5, 41)]
ax4 = axes[4].twinx()
hist, bins = np.histogram(conflicts['n_hat'], bins=bins)
ax4.scatter((bins[:-1]+bins[1:])/2, hist, s=10, color=cmap(0.6), marker='x', lw=0.8, alpha=0.6)
hist, bins = np.histogram(10**conflict_cases['avg_intensity'], bins=bins)
axes[4].scatter((bins[:-1]+bins[1:])/2, hist, s=15, color=cmap(0.), marker='+', lw=0.8, alpha=0.6)
axes[4].loglog()
ax4.loglog()
ylim = ax4.get_ylim()
ax4.set_ylim(ylim[0], ylim[1])
ylim = axes[4].get_ylim()
axes[4].set_ylim(ylim[0], ylim[1])
axes[4].set_xlabel('Unified intensity', labelpad=0)
axes[4].set_ylabel('Case frequency', labelpad=1)
ax4.set_ylabel('Moment frequency', labelpad=1)
axes[4].yaxis.label.set_color(cmap(0.))
ax4.yaxis.label.set_color(cmap(0.6))
axes[4].tick_params(axis='y', which='both', colors=cmap(0.), labelsize=8, pad=1)
ax4.tick_params(axis='y', which='both', colors=cmap(0.6), labelsize=8, pad=1)
ax4.xaxis.get_minor_locator().set_params(numticks=10)
hist_conflict_cases = hist.copy()

bins = [10**i for i in np.linspace(np.log10(1.1), 3.5, 41)]
hist, bins = np.histogram(conflicts['n_hat'], bins=bins)
bins = (bins[:-1]+bins[1:])/2
bins, hist = bins[hist_conflict_cases>0], hist[hist_conflict_cases>0]
tl = trendline(np.log10(bins), np.log10(hist))
bins = [10**i for i in np.linspace(np.log10(1.1), 3.5, 41)]
ax4.plot(bins, 10**tl(np.log10(bins)), ls='--', color=cmap(0.6), lw=0.8)
ax4.text(0.02, 0.15, "$y=10^{%.1f}x^{%.1f}$"%(tl[0],tl[1]), fontsize=8, ha='left', va='bottom', transform=ax4.transAxes, color=cmap(0.6))
bins = [10**i for i in np.linspace(np.log10(1.1), 3.5, 41)]
hist, bins = np.histogram(10**conflict_cases['avg_intensity'], bins=bins)
bins = (bins[:-1]+bins[1:])/2
bins, hist = bins[hist_conflict_cases>0], hist_conflict_cases[hist_conflict_cases>0]
tl = trendline(np.log10(bins), np.log10(hist))
bins = [10**i for i in np.linspace(np.log10(1.1), 3.5, 41)]
axes[4].plot(bins, 10**tl(np.log10(bins)), ls='--', color=cmap(0.), lw=0.8)
axes[4].text(0.02, 0.02, "$y=10^{%.1f}x^{%.1f}$"%(tl[0],tl[1]), fontsize=8, ha='left', va='bottom', transform=axes[4].transAxes, color=cmap(0.))

handle1, label1 = axes[3].get_legend_handles_labels()
handle2, label2 = ax3.get_legend_handles_labels()
ax_legend = fig.add_axes([0.5, 0., 0.5, 1])
ax_legend.axis('off')
ax_legend.legend(handle1+handle2, label1+label2, ncol=2, bbox_to_anchor=(0.5, -0.1), loc='lower center',
                 fontsize=8, frameon=False, handlelength=1., handletextpad=0.3, borderaxespad=0.3, borderpad=0.3)
ax_legend.set_title('Log-log plots of TTC and Unified intensity distribution', fontsize=9)

to_grayscale(fig)

In [ ]:
fig.savefig(fig_path + 'conflict_intensity_hist.pdf', dpi=600, bbox_inches='tight')